# Training

In [1]:
!git clone https://github.com/adhiiisetiawan/nbr-time_aware_item_weighting
%cd nbr-time_aware_item_weighting/
!pip install -r requirements.txt --quiet

Cloning into 'nbr-time_aware_item_weighting'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 103 (delta 7), reused 23 (delta 5), pack-reused 77
Receiving objects: 100% (103/103), 54.92 MiB | 19.68 MiB/s, done.
Resolving deltas: 100% (14/14), done.
Updating files: 100% (66/66), done.
/kaggle/working/nbr-time_aware_item_weighting
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
beatrix-jupyterlab 2023.621.222118 requires jupyter-server~=1.16, but you have jupyter-server 2.6.0 which is incompatible.
chex 0.1.82 requires numpy>=1.25.0, but you have numpy 1.22.4 which is incompatible.
fitter 1.6.0 requires pandas<3.0.0,>=2.0.3, but you have pandas 1.5.3 which is incompa

In [ ]:
import os
import sys
sys.path.append("..")
from nbr.preparation import Preprocess, save_split, Corpus
from nbr.trainer import NBRTrainer
from nbr.model import BPR, SLRC, NBRKNN, RepurchaseModule
import torch
import random
import numpy as np
import optuna
import warnings
warnings.filterwarnings("ignore")

In [ ]:
seed = 10
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

In [4]:
!mkdir /kaggle/working/emos-prep
!cp /kaggle/input/emos-prep/EMOS-prep.txt /kaggle/working/emos-prep

In [5]:
corpus_path = "/kaggle/working/emos-prep/"
dataset_name = "EMOS-prep"

preprocessor = Preprocess(corpus_path, dataset_name)
preprocessor.load_data(5, 10, filt=True)
save_split(corpus_path, dataset_name, preprocessor)

Before preprocessing: #users = 55596, #items = 3422, #clicks = 10524887 (#illegal records = 0)
After preprocessing: #users = 41748, #items = 3000, #clicks = 7054711
Saving dataset in /kaggle/working/emos-prep//data_EMOS-prep/...


In [6]:
corpus = Corpus(corpus_path, dataset_name)
corpus.load_data()

In [7]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=100,
    topk=10,
    early_stop_num=3
)

train dataset preparing...


100%|██████████| 41748/41748 [00:37<00:00, 1106.60it/s]


dev dataset preparing...


100%|██████████| 41748/41748 [01:09<00:00, 598.50it/s] 


test dataset preparing...


100%|██████████| 41748/41748 [01:12<00:00, 573.10it/s] 


In [8]:
slrc_best_params = {'batch_size': 256, 'lr': 0.00011201144001505824, 'l2_reg_coef': 0.00011498224071460201}

params = {
    "model": RepurchaseModule(
        item_num=corpus.n_items,
        avg_repeat_interval=corpus.total_avg_interval
    ),
    "batch_size": slrc_best_params["batch_size"],
    "lr": slrc_best_params["lr"],
    "l2_reg_coef": slrc_best_params["l2_reg_coef"]
}

trainer.init_hyperparams(**params)
trainer.train()

Epoch 1:


Batch loss = 0.323373: 100%|██████████| 25296/25296 [07:15<00:00, 58.08it/s]


Evaluation (dev):



100%|██████████| 41748/41748 [05:16<00:00, 131.81it/s]


 {'precision': 0.1587357478202549, 'recall': 0.36671096110509155, 'ndcg': 0.312801874828681}
Epoch 2:


...


100%|██████████| 41748/41748 [05:16<00:00, 131.81it/s]


 {'precision': 0.15651767749353263, 'recall': 0.3625929140892119, 'ndcg': 0.3078113943075695}
Epoch 4:



Batch loss = 0.350635:  41%|████▏     | 10465/25296 [03:03<04:07, 59.99it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Batch loss = 0.528102:  65%|██████▌   | 16443/25296 [04:47<02:33, 57.50it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Batch loss = 0.229695:  88%|████████▊ | 22305/25296 [06:30<00:51, 57.82it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid cra


 {'precision': 0.15586135862795825, 'recall': 0.3616493103344092, 'ndcg': 0.3069208922750109}


RepurchaseModule()

In [9]:
!cp /kaggle/working/nbr-time_aware_item_weighting/best_checkpoint.pth /kaggle/working/nbr-time_aware_item_weighting/best-emos.pth

In [10]:
dev_user_emb = trainer.get_predictions(mode="dev")
test_user_emb = trainer.get_predictions(mode="test")

100%|██████████| 41748/41748 [05:09<00:00, 134.73it/s]


In [11]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=None,
    topk=10,
    early_stop_num=None
)

train dataset preparing...


100%|██████████| 41748/41748 [00:37<00:00, 1106.60it/s]


dev dataset preparing...


100%|██████████| 41748/41748 [01:09<00:00, 604.40it/s] 


test dataset preparing...


100%|██████████| 41748/41748 [01:12<00:00, 578.07it/s] 


In [12]:
def objective(trial):
    params = {
        "model": NBRKNN(
            item_num=corpus.n_items,
            user_num=corpus.n_users,
            nearest_neighbors_num=trial.suggest_int("nearest_neighbors_num", low=1, high=200),
            alpha=trial.suggest_float("alpha", 0.0, 1.0, step=0.05),
            user_emb=dev_user_emb
        )
    }

    trainer.init_hyperparams(**params)
    metrics = trainer.evaluate(mode="dev")
    score = metrics["ndcg"]
    return score

sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=10)

[I 2023-11-07 05:21:13,669] A new study created in memory with name: no-name-32c9919b-b60b-423b-b743-e95b1b202d38
100%|██████████| 41748/41748 [06:20<00:00, 109.61it/s]
[I 2023-11-07 05:27:51,073] Trial 0 finished with value: 0.2882812321218112 and parameters: {'nearest_neighbors_num': 155, 'alpha': 0.0}. Best is trial 0 with value: 0.2882812321218112.
100%|██████████| 41748/41748 [06:22<00:00, 109.15it/s]
[I 2023-11-07 05:34:29,910] Trial 1 finished with value: 0.3215161611915018 and parameters: {'nearest_neighbors_num': 127, 'alpha': 0.75}. Best is trial 1 with value: 0.3215161611915018.
100%|██████████| 41748/41748 [06:29<00:00, 107.06it/s]
[I 2023-11-07 05:41:17,952] Trial 2 finished with value: 0.3099212069382438 and parameters: {'nearest_neighbors_num': 100, 'alpha': 0.2}. Best is trial 1 with value: 0.3215161611915018.
100%|██████████| 41748/41748 [05:40<00:00, 122.69it/s]
[I 2023-11-07 05:47:14,152] Trial 3 finished with value: 0.3219344639877249 and parameters: {'nearest_neigh

In [13]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=None,
    topk=10,
    early_stop_num=None
)

params = {
    "model": NBRKNN(
        item_num=corpus.n_items,
        user_num=corpus.n_users,
        nearest_neighbors_num=study.best_params["nearest_neighbors_num"],
        alpha=study.best_params["alpha"],
        user_emb=dev_user_emb
    )
}

params["model"].set_emb(test_user_emb)

trainer.init_hyperparams(**params)
metrics = trainer.evaluate(mode="test")
metrics

train dataset preparing...


100%|██████████| 41748/41748 [00:37<00:00, 1101.93it/s]


dev dataset preparing...


100%|██████████| 41748/41748 [01:08<00:00, 611.02it/s] 


test dataset preparing...


100%|██████████| 41748/41748 [05:36<00:00, 124.12it/s]


{'precision': 0.1683290217495449,
 'recall': 0.3837650979640677,
 'ndcg': 0.3277789311962349}

In [14]:
import pandas as pd

data = pd.read_csv('/kaggle/input/emos-prep/EMOS.csv')

In [17]:
def inference(user, topk):
    n_items = corpus.n_items

    items_scores = trainer.model.predict_for_user(user).view(-1, n_items)
    top_items = torch.topk(items_scores, k=topk, dim=1, sorted=True).indices
    top_score = [float(i) for i in items_scores[0]]

    print(sorted(data['ship_to_id_emos'].unique())[user])
    print("Item  | Score")
    print('-------------')
    for i in top_items[0]:
        print("{:5} | {:<5}".format(sorted(data['prod_id'].unique())[int(i)], round(top_score[i], 3)))
    print()

top_k = 5

for i in [1, 12, 123]:
    inference(i, top_k)

34746
Item  | Score
-------------
10137 | 1.194
10199 | 0.625
10197 | 0.614
10194 | 0.569
10154 | 0.499

34890
Item  | Score
-------------
10165 | 1.253
10154 | 1.108
10157 | 1.015
10137 | 0.825
10152 | 0.68 

36133
Item  | Score
-------------
10156 | 1.54 
10152 | 1.408
10150 | 1.364
10154 | 1.184
10226 | 0.957



# Load

In [1]:
# !git clone https://github.com/adhiiisetiawan/nbr-time_aware_item_weighting
# %cd nbr-time_aware_item_weighting/
!pip install -r ../requirements.txt --quiet

DEPRECATION: celery 4.4.1 has a non-standard dependency specifier pytz>dev. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
import sys
sys.path.append("..")
from nbr.preparation import Preprocess, save_split, Corpus
from nbr.trainer import NBRTrainer
from nbr.model import NBRKNN, RepurchaseModule
import torch
import random
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [3]:
seed = 10
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

In [2]:
corpus_path = "data/"
dataset_name = "EMOS-prep"

preprocessor = Preprocess(corpus_path, dataset_name)
preprocessor.load_data(5, 10, filt=True)
save_split(corpus_path, dataset_name, preprocessor)

corpus = Corpus(corpus_path, dataset_name)
corpus.load_data()

Before preprocessing: #users = 55596, #items = 3422, #clicks = 10524887 (#illegal records = 0)
After preprocessing: #users = 41748, #items = 3000, #clicks = 7054711
Saving dataset in data//data_EMOS-prep/...


In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=100,
    topk=10,
    early_stop_num=3
)

slrc_best_params = {'batch_size': 256, 'lr': 0.00011201144001505824, 'l2_reg_coef': 0.00011498224071460201}

params = {
    "model": RepurchaseModule(
        item_num=corpus.n_items,
        avg_repeat_interval=corpus.total_avg_interval
    ),
    "batch_size": slrc_best_params["batch_size"],
    "lr": slrc_best_params["lr"],
    "l2_reg_coef": slrc_best_params["l2_reg_coef"]
}

trainer.init_hyperparams(**params)
with open("best_checkpoint.pth", "rb") as f:
    checkpoint = torch.load(f)
    trainer.model.load_state_dict(checkpoint)

train dataset preparing...


100%|██████████| 41748/41748 [00:37<00:00, 1112.72it/s]


dev dataset preparing...


100%|██████████| 41748/41748 [03:01<00:00, 230.06it/s] 


test dataset preparing...


100%|██████████| 41748/41748 [02:47<00:00, 249.14it/s] 


In [ ]:
test_user_emb = trainer.get_predictions(mode="test")

100%|██████████| 41748/41748 [11:58<00:00, 58.10it/s]


In [7]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=None,
    topk=10,
    early_stop_num=None
)

best_study = {
    'nearest_neighbors_num': 100,
    'alpha': 1
}

params = {
    "model": NBRKNN(
        item_num=corpus.n_items,
        user_num=corpus.n_users,
        nearest_neighbors_num=best_study["nearest_neighbors_num"],
        alpha=best_study["alpha"],
        user_emb=test_user_emb
    )
}

trainer.init_hyperparams(**params)
trainer.evaluate(mode='test')

train dataset preparing...


100%|██████████| 41748/41748 [00:39<00:00, 1069.84it/s]


dev dataset preparing...


100%|██████████| 41748/41748 [06:27<00:00, 107.66it/s] 


test dataset preparing...


100%|██████████| 41748/41748 [16:01<00:00, 43.40it/s] 


{'precision': 0.1683290217495449,
 'recall': 0.3837650979640677,
 'ndcg': 0.327779081801798}

In [8]:
import pandas as pd

data = pd.read_csv('data/EMOS.csv')

In [1]:
def inference(user, topk):
    n_items = corpus.n_items

    items_scores = trainer.model.predict_for_user(user).view(-1, n_items)
    top_items = torch.topk(items_scores, k=topk, dim=1, sorted=True).indices
    top_score = [float(i) for i in items_scores[0]]

    print(sorted(data['ship_to_id_emos'].unique())[user])
    print("Item  | Score")
    print('-------------')
    for i in top_items[0]:
        print("{:5} | {:<5}".format(sorted(data['prod_id'].unique())[int(i)], round(top_score[i], 3)))
    print()

top_k = 10

for i in [1, 12, 123]:
    inference(i, top_k)

34746
Item  | Score
-------------
10137 | 1.194
10199 | 0.625
10197 | 0.614
10194 | 0.569
10154 | 0.499

34890
Item  | Score
-------------
10165 | 1.253
10154 | 1.108
10157 | 1.015
10137 | 0.825
10152 | 0.68 

36133
Item  | Score
-------------
10156 | 1.54 
10152 | 1.408
10150 | 1.364
10154 | 1.184
10226 | 0.957

